# Convert SQLite outputs to parquet files with cytotable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# type of file output from cytotable (currently only parquet)
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path("../2.feature_extraction/sqlite_outputs")

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 6 plates in this dataset. Below are the names:
BR00143980
BR00143977
BR00143976
BR00143979
BR00143981
BR00143978


## Convert SQLite to parquet files

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(
        f"{output_dir}/converted_profiles/{file_path.stem}_converted.parquet"
    )
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # Merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000,
    )

print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: BR00143980
Starting conversion with cytotable for plate: BR00143977
Starting conversion with cytotable for plate: BR00143976
Starting conversion with cytotable for plate: BR00143979
Starting conversion with cytotable for plate: BR00143981
Starting conversion with cytotable for plate: BR00143978
All plates have been converted with cytotable!


# Load in converted profiles to update

In [4]:
# Directory with converted profiles
converted_dir = pathlib.Path(f"{output_dir}/converted_profiles")

# Define the list of columns to prioritize and prefix
prioritized_columns = [
    "Nuclei_Location_Center_X",
    "Nuclei_Location_Center_Y",
    "Cells_Location_Center_X",
    "Cells_Location_Center_Y",
    "Image_Count_Cells"
]

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)

    # If any, drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])
    
    # Rearrange columns and add "Metadata" prefix in one line
    df = df[
        prioritized_columns + [col for col in df.columns if col not in prioritized_columns]
    ].rename(
        columns=lambda col: "Metadata_" + col if col in prioritized_columns else col
    )

    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)

## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [5]:
converted_df = pd.read_parquet(
    f"./data/converted_profiles/{plate_names[0]}_converted.parquet"
)

print(converted_df.shape)
converted_df.head()

(594141, 3009)


,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,...,Nuclei_Texture_Variance_CorrER_3_02_256,Nuclei_Texture_Variance_CorrER_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrRNA_3_00_256,Nuclei_Texture_Variance_CorrRNA_3_01_256,Nuclei_Texture_Variance_CorrRNA_3_02_256,Nuclei_Texture_Variance_CorrRNA_3_03_256
0,888.255102,52.769679,909.879420,75.402259,78,1436,BR00143980,5,F16,1,...,31.824278,32.938104,254.835846,232.326666,252.985878,234.215995,10.097313,10.213155,9.726383,9.673958
1,283.495951,48.856275,258.168863,60.427512,167,1350,BR00143980,9,H15,1,...,147.654152,159.346124,668.249576,583.078253,590.721431,638.903869,83.641604,73.628611,76.593809,83.423347
2,89.630769,29.233566,88.956300,28.393299,176,1365,BR00143980,6,J15,1,...,202.211024,170.945610,1106.306179,1052.228951,1146.478867,1082.087840,148.026652,143.908656,160.392433,144.518192
3,990.509960,23.270916,986.840399,20.955112,423,1368,BR00143980,9,J15,1,...,12.172090,11.820776,59.347112,55.371570,63.239796,64.462703,5.364327,5.481792,5.356909,5.133591
4,527.035887,264.812803,527.745098,265.026403,64,1370,BR00143980,2,K15,1,...,33.237022,34.945281,84.415866,82.661372,78.843635,79.200450,15.187183,15.093495,15.067460,15.174425
